In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('modeldata.csv')

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
df.drop(columns='Unnamed: 0',inplace=True)

In [10]:
df['loan_status'].value_counts()/63999

0    0.759027
1    0.236769
Name: loan_status, dtype: float64

In [ ]:
# rfe for random forest

In [15]:
from scipy.stats import randint as randint
params2={
'n_estimators': randint(1,300),
'criterion':['gini','entropy'],
    
    'max_depth':randint(1,200),
    'min_samples_split':randint(1,150),
    'min_samples_leaf':randint(1,200)}

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x = df.drop(columns=['loan_status'])
y = df['loan_status']

In [8]:

>>> from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=2)

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rfc = RandomForestClassifier()

In [13]:
from sklearn.model_selection import RandomizedSearchCV

In [16]:
rsvrf = RandomizedSearchCV(rfc,params2,n_jobs=-1,n_iter=100,cv=5).fit(x_train,y_train)

In [17]:
rsvrf.best_params_

{'criterion': 'entropy',
 'max_depth': 169,
 'min_samples_leaf': 1,
 'min_samples_split': 22,
 'n_estimators': 221}

In [19]:
rf1 = RandomForestClassifier(**rsvrf.best_params_).fit(x_train,y_train)

In [21]:
y_tr_pred = rf1.predict(x_train)

In [9]:
from sklearn.metrics import accuracy_score,classification_report

In [23]:
accuracy_score(y_train,y_tr_pred)

0.9490135364144193

In [24]:
print(classification_report(y_train,y_tr_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     36479
           1       0.96      0.82      0.88     11318

    accuracy                           0.95     47797
   macro avg       0.95      0.90      0.93     47797
weighted avg       0.95      0.95      0.95     47797



In [25]:
y_te_pred = rf1.predict(x_test)

In [26]:
accuracy_score(y_test,y_te_pred)

0.9012113224126028

In [27]:
print(classification_report(y_test,y_te_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94     12098
           1       0.90      0.66      0.76      3835

    accuracy                           0.90     15933
   macro avg       0.90      0.82      0.85     15933
weighted avg       0.90      0.90      0.90     15933



In [28]:
from sklearn.feature_selection import RFE

In [29]:
rfe = RFE(rf1,50).fit(x_train,y_train)

In [30]:
(rfe.ranking_)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5,
       1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1,
       1, 1, 1, 1, 9, 1, 1, 1, 1, 3, 1, 2, 1, 1])

In [31]:
pd.DataFrame([x_train.columns,rfe.ranking_]).T

,0,1
0,member_id,1
1,loan_amnt,1
2,funded_amnt_inv,1
3,term,1
4,int_rate,1
5,emp_length,1
6,annual_inc,1
7,dti,1
8,delinq_2yrs,1
9,inq_last_6mths,1


In [32]:
rfe_df = df.drop(columns=['acc_now_delinq','grade_G','home_ownership_NONE','home_ownership_OTHER','purpose_educational','application_type_JOINT','Nurse','Owner'])

In [33]:
rfe_df.shape

(63730, 51)

In [34]:
xrfe = rfe_df.drop(columns=['loan_status'])
yrfe = rfe_df['loan_status']

In [35]:
xrfe_train,xrfe_test,yrfe_train,yrfe_test = train_test_split(xrfe,yrfe,test_size=0.25,random_state=1)

In [36]:
rf2 = RandomForestClassifier(**rsvrf.best_params_).fit(xrfe_train,yrfe_train)

In [37]:
y_tr1_pred = rf2.predict(xrfe_train)

In [38]:
accuracy_score(y_train,y_tr1_pred)

0.9520262777998619

In [41]:
print(classification_report(yrfe_train,y_tr1_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     36479
           1       0.96      0.84      0.89     11318

    accuracy                           0.95     47797
   macro avg       0.95      0.91      0.93     47797
weighted avg       0.95      0.95      0.95     47797



In [40]:
y_ta1_pred = rf2.predict(xrfe_test)

In [42]:
accuracy_score(yrfe_test,y_ta1_pred)

0.9076131299817988

In [43]:
print(classification_report(yrfe_test,y_ta1_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94     12098
           1       0.90      0.69      0.78      3835

    accuracy                           0.91     15933
   macro avg       0.91      0.83      0.86     15933
weighted avg       0.91      0.91      0.90     15933



In [44]:
rfe1 = RFE(rf1,55).fit(x_train,y_train)

In [45]:
pd.DataFrame([x_train.columns,rfe1.ranking_]).T

,0,1
0,member_id,1
1,loan_amnt,1
2,funded_amnt_inv,1
3,term,1
4,int_rate,1
5,emp_length,1
6,annual_inc,1
7,dti,1
8,delinq_2yrs,1
9,inq_last_6mths,1


In [46]:
rfe_df1 = df.drop(columns=['home_ownership_NONE','purpose_educational','application_type_JOINT'])

In [47]:
xrfe1 = rfe_df1.drop(columns=['loan_status'])
yrfe1 = rfe_df1['loan_status']

In [48]:
xrfe_train1,xrfe_test1,yrfe_train1,yrfe_test1 = train_test_split(xrfe1,yrfe1,test_size=0.25,random_state=1)

In [49]:
rf3 = RandomForestClassifier(**rsvrf.best_params_).fit(xrfe_train1,yrfe_train1)

In [50]:
y_tr2_pred = rf3.predict(xrfe_train1)

In [51]:
accuracy_score(yrfe_train1,y_tr2_pred)

0.950498985291964

In [52]:
print(classification_report(yrfe_train1,y_tr2_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     36479
           1       0.96      0.83      0.89     11318

    accuracy                           0.95     47797
   macro avg       0.95      0.91      0.93     47797
weighted avg       0.95      0.95      0.95     47797



In [53]:
y_ta_pred2 = rf3.predict(xrfe_test1)

In [54]:
accuracy_score(y_test,y_ta_pred2)

0.9035963095462248

In [56]:
print(classification_report(y_test,y_ta_pred2))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94     12098
           1       0.90      0.67      0.77      3835

    accuracy                           0.90     15933
   macro avg       0.90      0.82      0.85     15933
weighted avg       0.90      0.90      0.90     15933



In [ ]:
# logistic reg for rfe

In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
lr = LogisticRegression()

In [60]:
parmns3={
'penalty':['l1', 'l2', 'elasticnet', 'none'],
'tol':[0.0001,0.00001,0.000001,0.0000001],
'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [61]:
logrsv = RandomizedSearchCV(lr,parmns3,n_iter=50,n_jobs=-1,cv=5).fit(x_train,y_train)

In [62]:
lr1 = LogisticRegression(**logrsv.best_params_).fit(x_train,y_train)

/Users/zameershaik/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [63]:
y_train_predloghp = lr1.predict(x_train)

In [64]:
accuracy_score(y_train,y_train_predloghp)

0.8831307404230391

In [65]:
print(classification_report(y_train,y_train_predloghp))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     36479
           1       0.82      0.65      0.72     11318

    accuracy                           0.88     47797
   macro avg       0.86      0.80      0.82     47797
weighted avg       0.88      0.88      0.88     47797



In [66]:
y_test_predloghp = lr1.predict(x_test)

In [67]:
print(accuracy_score(y_test,y_test_predloghp))

0.8805623548609803


In [68]:
print(classification_report(y_test,y_test_predloghp))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92     12098
           1       0.81      0.65      0.72      3835

    accuracy                           0.88     15933
   macro avg       0.86      0.80      0.82     15933
weighted avg       0.88      0.88      0.88     15933



In [69]:
rfel = RFE(lr1,50).fit(x_train,y_train)

In [70]:
pd.DataFrame([x_train.columns,rfe1.ranking_]).T

,0,1
0,member_id,1
1,loan_amnt,1
2,funded_amnt_inv,1
3,term,1
4,int_rate,1
5,emp_length,1
6,annual_inc,1
7,dti,1
8,delinq_2yrs,1
9,inq_last_6mths,1


In [71]:
lr1_df = df.drop(columns=['home_ownership_NONE','purpose_educational','application_type_JOINT'])

In [72]:
xrfe3 = lr1_df.drop(columns=['loan_status'])
yrfe3 = lr1_df['loan_status']

In [73]:
xrfe_train,xrfe_test,yrfe_train,yrfe_test = train_test_split(xrfe3,yrfe3,test_size=0.25,random_state=1)

In [74]:
lr2 = LogisticRegression(**logrsv.best_params_).fit(xrfe_train,yrfe_train)

In [75]:
ypred_trl = lr2.predict(xrfe_train)

In [76]:
accuracy_score(yrfe_train,ypred_trl)

0.882607695043622

In [77]:
print(classification_report(yrfe_train,ypred_trl))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     36479
           1       0.82      0.65      0.72     11318

    accuracy                           0.88     47797
   macro avg       0.86      0.80      0.82     47797
weighted avg       0.88      0.88      0.88     47797



In [78]:
ypred_te1 = lr2.predict(xrfe_test)

In [79]:
accuracy_score(yrfe_test,ypred_te1)

0.8801230151258395

In [81]:
print(classification_report(yrfe_test,ypred_te1))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92     12098
           1       0.81      0.65      0.72      3835

    accuracy                           0.88     15933
   macro avg       0.85      0.80      0.82     15933
weighted avg       0.88      0.88      0.88     15933



In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
for i in range(3,20):
    knn = KNeighborsClassifier(i).fit(x_train,y_train)
    y_test_pred = knn.predict(x_test)
    print('No of Neighbours:',i)
    print(accuracy_score(y_test,y_test_pred))
    print(classification_report(y_test,y_test_pred))

No of Neighbours: 3
0.7846607669616519
              precision    recall  f1-score   support

           0       0.84      0.88      0.86     12098
           1       0.56      0.49      0.52      3835

    accuracy                           0.78     15933
   macro avg       0.70      0.68      0.69     15933
weighted avg       0.78      0.78      0.78     15933

No of Neighbours: 4
0.8024226448252055
              precision    recall  f1-score   support

           0       0.83      0.94      0.88     12098
           1       0.65      0.38      0.48      3835

    accuracy                           0.80     15933
   macro avg       0.74      0.66      0.68     15933
weighted avg       0.79      0.80      0.78     15933

No of Neighbours: 5
0.7988451641247725
              precision    recall  f1-score   support

           0       0.84      0.90      0.87     12098
           1       0.60      0.48      0.53      3835

    accuracy                           0.80     15933
   macro av

In [ ]:
# 54
# 17



In [16]:
from sklearn.model_selection import RandomizedSearchCV

In [11]:
from scipy.stats import randint as ri

In [14]:
    params3={
  
    'weights':['uniform','distance'],
    
    'leaf_size':ri(1,100),
    'p':ri(1,5),
    }

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
ss = StandardScaler()

In [24]:
ssx_train = ss.fit_transform(x_train)
ssx_test = ss.fit_transform(x_test)

In [13]:
knn = KNeighborsClassifier()

In [ ]:
knnhp1 = RandomizedSearchCV(knn,params3).fit(ssx_train,y_train)

In [18]:
alg =['auto', 'ball_tree', 'kd_tree', 'brute']

In [20]:
from sklearn.metrics import accuracy_score,classification_report

In [21]:
for i in alg:
    knn12 = KNeighborsClassifier(n_neighbors=54,algorithm=i,**knnhp1.best_params_).fit(ssx_train,y_train)
    y_hpknn_trpred = knn12.predict(ssx_train)
    accuracy_score(y_train,y_hpknn_trpred)
    print(classification_report(y_train,y_hpknn_trpred))
    y_hpknn_trpred1 = knn12.predict(ssx_test)
    accuracy_score(y_test,y_hpknn_trpred1)
    print(classification_report(y_test,y_hpknn_trpred1))
    
    
    

              precision    recall  f1-score   support

           0       0.84      0.95      0.89     36479
           1       0.74      0.42      0.54     11318

    accuracy                           0.83     47797
   macro avg       0.79      0.69      0.72     47797
weighted avg       0.82      0.83      0.81     47797

              precision    recall  f1-score   support

           0       0.84      0.95      0.89     12098
           1       0.73      0.43      0.54      3835

    accuracy                           0.82     15933
   macro avg       0.79      0.69      0.72     15933
weighted avg       0.81      0.82      0.81     15933

              precision    recall  f1-score   support

           0       0.84      0.95      0.89     36479
           1       0.74      0.42      0.54     11318

    accuracy                           0.83     47797
   macro avg       0.79      0.69      0.72     47797
weighted avg       0.82      0.83      0.81     47797

              preci

In [ ]:
knnhp1 = KNeighborsClassifier(n_neighbors=54,algorithm=i,**knnhp1.best_params_).fit(x_train,y_train)